*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will predict healthcare costs using a regression algorithm.

You are given a dataset that contains information about different people including their healthcare costs. Use the data to predict healthcare costs based on new data.

The first two cells of this notebook import libraries and the data.

Make sure to convert categorical data to numbers. Use 80% of the data as the `train_dataset` and 20% of the data as the `test_dataset`.

`pop` off the "expenses" column from these datasets to create new datasets called `train_labels` and `test_labels`. Use these labels when training your model.

Create a model and train it with the `train_dataset`. Run the final cell in this notebook to check your model. The final cell will use the unseen `test_dataset` to check how well the model generalizes.

To pass the challenge, `model.evaluate` must return a Mean Absolute Error of under 3500. This means it predicts health care costs correctly within $3500.

The final cell will also predict expenses using the `test_dataset` and graph the results.

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
# Exploring the data

# dataset.info()
# dataset['sex'].unique()
# dataset['smoker'].unique()
# dataset['region'].unique()


In [ ]:
# Selecting the labels and features of the data

labels = dataset.iloc[:, -1]
features = dataset.iloc[:, 0:-1]
# labels.head()
features.head()

In [ ]:
#  Converting the categorical data to numbers

features = pd.get_dummies(features)
features.head()

In [ ]:
# Splitting the dataset into test and train data

from sklearn.model_selection import train_test_split

train_dataset, test_dataset, train_labels, test_labels = train_test_split(features, labels, random_state=25, test_size=0.2)


In [ ]:
# Standardizing the numerical features

from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

numerical_features = features.select_dtypes(include=['float64', 'int64'])
numerical_columns = numerical_features.columns

ct = ColumnTransformer([("only numeric", StandardScaler(), numerical_columns)], remainder='passthrough')
train_dataset = ct.fit_transform(train_dataset)
test_dataset = ct.transform(test_dataset)

In [ ]:
# Building the model

model = keras.models.Sequential()

# Creating the input layer to the network model with the shape corresponding to the number of features in the dataset.
input = layers.InputLayer(input_shape=(features.shape[1],))

model.add(input)

model.add(layers.Dense(64, activation='relu')) # adding a hidden layer with 64 hidden units

model.add(layers.Dense(1)) # adding an output layer with one neuron since a single output is needed for a regression prediction

model.summary()

In [ ]:
# Initializing an Adam optimizer with the learning rate equal to 0.1
opt = keras.optimizers.Adam(learning_rate=0.01)

# Compiling a model with the Mean Squared Error loss, Mean Absolute Error  and Mean Squared Error learning metrics, and an optimizer:
model.compile(loss='mse', metrics=['mae','mse'], optimizer=opt)

# Fitting the model
model.fit(train_dataset, train_labels, epochs = 30, batch_size = 3, verbose = 0)




In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
